In [2]:
import tensorflow as tf
import keras
import os

/home/bis/miniconda3/envs/TF1.10/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bis/miniconda3/envs/TF1.10/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bis/miniconda3/envs/TF1.10/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bis/miniconda3/envs/TF1.10/lib/python3.6/site-

In [3]:
import numpy as np

In [4]:
# Load the saved Yaml model
from keras.models import model_from_yaml

# Model params

In [13]:
model_spec = 'frag/prosit1/'
model_irt = 'irt/model_irt_prediction'

spec_MODEL_NAME = "model.yml"
irt_MODEL_NAME = 'model.yaml'
CONFIG_NAME = "config.yml"

spec_model_path = os.path.join(model_spec, spec_MODEL_NAME)
irt_model_path = os.path.join(model_irt, irt_MODEL_NAME)

print(spec_model_path, irt_model_path)

frag/prosit1/model.yml irt/model_irt_prediction/model.yaml


# spec

In [61]:
#Read  the model architecture from  YAML file
with open(spec_model_path, 'r') as yaml_file:
    yaml_savedModel=yaml_file.read()

In [32]:
from layers import *
from losses import *

In [62]:
model_spec = model_from_yaml(yaml_savedModel, 
                             custom_objects={"Attention": Attention})

In [63]:
model_spec.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
peptides_in (InputLayer)        (None, 30)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 32)       704         peptides_in[0][0]                
__________________________________________________________________________________________________
encoder1 (Bidirectional)        (None, 30, 512)      445440      embedding[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 30, 512)      0           encoder1[0][0]                   
__________________________________________________________________________________________________
collision_

In [49]:
param_spec = keras.models.load_model("frag/prosit1/weight_32_0.10211.hdf5",
                                    custom_objects={"Attention": Attention,
                                                   "masked_spectral_distance": masked_spectral_distance}
                                    )

/home/bis/miniconda3/envs/TF1.10/lib/python3.6/site-packages/keras/engine/saving.py:305: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [51]:
param_spec.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
peptides_in (InputLayer)        (None, 30)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 32)       704         peptides_in[0][0]                
__________________________________________________________________________________________________
encoder1 (Bidirectional)        (None, 30, 512)      445440      embedding[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 30, 512)      0           encoder1[0][0]                   
__________________________________________________________________________________________________
collision_

# iRT

In [54]:
#Read  the model architecture from  YAML file
with open(irt_model_path, 'r') as yaml_file:
    yaml_savedModel=yaml_file.read()

In [56]:
model_irt = model_from_yaml(yaml_savedModel, custom_objects={"Attention": Attention})

In [57]:
model_irt.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
peptides_in (InputLayer)     (None, 30)                0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 32)            704       
_________________________________________________________________
encoder1 (Bidirectional)     (None, 30, 512)           445440    
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
encoder2 (CuDNNGRU)          (None, 30, 512)           1575936   
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
encoder_att (Attention)      (None, 512)               542       
__________

In [59]:
param_irt = keras.models.load_model("irt/model_irt_prediction/weight_66_0.00796.hdf5",
                                    custom_objects={"Attention": Attention}
                                    )

In [60]:
param_irt.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
peptides_in (InputLayer)     (None, 30)                0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 32)            704       
_________________________________________________________________
encoder1 (Bidirectional)     (None, 30, 512)           445440    
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
encoder2 (CuDNNGRU)          (None, 30, 512)           1575936   
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
encoder_att (Attention)      (None, 512)               542       
__________

# iRT data

In [7]:
import h5py
filename = "../data/iRT/PrositIRT_data.hdf5"

In [88]:
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    irt_data = dict()
    for k in list(f.keys()):
        data = np.array(f[k])
        print(k, data.shape)
        
        # Get the data
        irt_data[k] = data

Keys: <KeysViewHDF5 ['X_holdout', 'X_train', 'X_val', 'Y_holdout', 'Y_train', 'Y_val']>
X_holdout (169339, 30)
X_train (349136, 1)
X_val (87455, 30)
Y_holdout (169339, 1)
Y_train (349136, 30)
Y_val (87455, 1)


# MS2 data (5GB)

In [9]:
filename = "../data/frag/holdout_hcd.hdf5"

In [10]:
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    frag_hold_data = dict()
    for k in list(f.keys()):
        data = np.array(f[k])
        print(k, data.shape)
        
        # Get the data
        frag_hold_data[k] = data

Keys: <KeysViewHDF5 ['collision_energy', 'collision_energy_aligned', 'collision_energy_aligned_normed', 'intensities_raw', 'masses_pred', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer']>
collision_energy (754215, 1)
collision_energy_aligned (754215, 1)
collision_energy_aligned_normed (754215, 1)
intensities_raw (754215, 174)
masses_pred (754215, 174)
masses_raw (754215, 174)
method (754215, 3)
precursor_charge_onehot (754215, 6)
rawfile (754215,)
reverse (754215, 1)
scan_number (754215, 1)
score (754215, 1)
sequence_integer (754215, 30)


In [84]:
filename = "../data/frag/traintest_hcd.hdf5"

In [85]:
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    frag_train_data = dict()
    for k in list(f.keys()):
        data = np.array(f[k])
        print(k, data.shape)
        
        # Get the data
        frag_train_data[k] = data

Keys: <KeysViewHDF5 ['collision_energy', 'collision_energy_aligned', 'collision_energy_aligned_normed', 'intensities_raw', 'masses_pred', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer', 'sequence_onehot']>
collision_energy (6787933, 1)
collision_energy_aligned (6787933, 1)
collision_energy_aligned_normed (6787933, 1)
intensities_raw (6787933, 174)
masses_pred (6787933, 174)
masses_raw (6787933, 174)
method (6787933, 3)
precursor_charge_onehot (6787933, 6)
rawfile (6787933,)
reverse (6787933, 1)
scan_number (6787933, 1)
score (6787933, 1)
sequence_integer (6787933, 30)
sequence_onehot (6787933, 30, 22)
